In [82]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import math

from os import listdir, walk
from os.path import isfile, join

from datetime import datetime, timedelta

In [3]:
from sklearn.preprocessing import MinMaxScaler

# Define Timestamp Methods

In [4]:
def calc_sec(time):
    hms = time.split(':')
    hms = [float(x) for x in hms]
    sec = hms[2] + hms[1]*60 + hms[0]*3600
    sec = round(sec,3)
    return sec

In [5]:
def calc_ts(sec):
    ts = ''
    hr = int(sec/3600)
    mn = int((sec - (hr*3600))/60)
    sc = sec - (hr*3600) - (mn*60)
    sc = round(sc,3)
    ts += str(hr) + ':' + str(mn) + ':' + str(sc)
    # print(ts)
    return ts

In [6]:
def calc_t_period(dates,secs):
    t_period = []
    
    start_sec = secs[0]
    prev_sec = secs[0]
    prev_date = dates[0]

    for i in range(len(secs)):
        curr_sec = secs[i]
        diff_sec = curr_sec - prev_sec
        curr_date = dates[i]
        
        if((diff_sec>3.0) and (curr_date==prev_date)):
            t_period.append([curr_date,start_sec,prev_sec])
            start_sec = curr_sec
        elif(curr_date!=prev_date):
            t_period.append([prev_date,start_sec,prev_sec])
            start_sec = curr_sec
            prev_date = curr_date
        elif(i==len(secs)-1):
            t_period.append([curr_date,start_sec,curr_sec])

        prev_sec = curr_sec
    
    return t_period

# Load Dataset

In [7]:
# Retrieve file directories from Google Drive
mypath = '../../DDC_Data/raw/'
basepath = '../../'

dir_ = [f for f in walk(mypath)]
# print(dir_)

dir = list(dir_[0])
dir[1] = sorted(dir[1])

outer_path = dir[0]
sub_path = dir[1]

folders = [join(outer_path,d) for d in sub_path]

files = []
for fd in folders:
    temp_f = [f for f in listdir(fd) if isfile(join(fd, f)) and f[-3:]=='csv' and f[5:9]!='data' and f[:4]==fd[-4:]]
    temp_f = sorted(temp_f)

## Retrieve All Timestamp Periods from a File

In [8]:
all_subjects = []

for i in range(1001,1015):
    all_subjects.append(str(i))

for i in range(3001,3007):
    all_subjects.append(str(i))

In [67]:
def load_timer(subject_id):
  # Configure starting and ending time values
    sid_dir = mypath + subject_id
    sid_files = [f for f in listdir(sid_dir) if 'history_amdtimer' in f]

    sid_filepath = sid_dir + '/' + sid_files[0]

    # Timestamp periods dataframe
    timer_df = pd.read_csv(sid_filepath, header=None, names=['sid','raw_label', 'timestamp', 'duration','label'])

    filtered_timer = [i for i in timer_df['sid'] if i==int(subject_id)]

    timer_filt = timer_df[timer_df['sid'].isin(filtered_timer)]
    timer_filt = timer_filt.reset_index(drop=True)
    
    timer_label = []
    
    for i in range(len(timer_filt)):
        if(timer_filt.loc[i]['raw_label']=='upstairs' or 
          timer_filt.loc[i]['raw_label']=='downstairs'):
            timer_label.append('walk')
        else:
            timer_label.append(timer_filt.loc[i]['raw_label'])

    timer_filt['label'] = pd.Series(timer_label)
    
    datetime_format = '%Y-%m-%d %H:%M:%S.%f'
    timer_filt['time_start'] = timer_filt['timestamp'].apply(lambda x: datetime.strptime(x, datetime_format))
    
    time_format = '%H:%M:%S'
    zero_date = datetime(1900, 1, 1)
    
    timer_filt['duration'] = timer_filt['duration'].apply(lambda x: datetime.strptime(x, time_format)-zero_date)
    
    for i in range(timer_filt.shape[0]):
        timer_filt.loc[i, 'time_end'] = timer_filt.loc[i, 'time_start'] + timer_filt.loc[i, 'duration']

#     print(timer_filt)
    
    return timer_filt

## Create Dataframe of ACC and HR

In [52]:
def load_acc(subject_id, start_time, end_time):
    # Load accelerations
    acc_filepath = mypath + '/' + subject_id + '/' + subject_id + '-log_acc.csv'

    df = pd.read_csv(acc_filepath, header=None, names=['x','y','z','timestamp'])
    
    datetime_format = '%Y-%m-%d %H:%M:%S.%f'
    df['timestamp'] = df['timestamp'].apply(lambda x: datetime.strptime(x, datetime_format))

    filtered = [r for r in df['timestamp'] if r>=start_time and r<=end_time]

    df_filt = df[df['timestamp'].isin(filtered)]
    df_filt = df_filt.reset_index(drop=True)

    df_filt['ID'] = pd.Series([subject_id for i in range(len(df_filt))])
    
    cols = ['ID','timestamp','x','y','z']
    df_filt = df_filt[cols]

    return df_filt

In [56]:
def load_hr(subject_id, start_time, end_time):
    # Load heart rate
    hr_filepath = mypath + '/' + subject_id + '/' + subject_id + '-log_hr.csv'

    df2 = pd.read_csv(hr_filepath, header=None, names=['hr','timestamp'])
    
    datetime_format = '%Y-%m-%d %H:%M:%S.%f'
    df2['timestamp'] = df2['timestamp'].apply(lambda x: datetime.strptime(x, datetime_format))

    filtered = [r for r in df2['timestamp'] if r>=start_time and r<=end_time]

    df_hr = df2[df2['timestamp'].isin(filtered)]
    df_hr = df_hr.reset_index(drop=True)

    cols = ['timestamp','hr']
    df_hr = df_hr[cols]

    return df_hr

In [59]:
def merge_acc_and_hr(df_filt, df_hr):
    # Fill in missing HRs
    hr_cnt = 0

    for i in range(len(df_filt)):
        hr_time = df_hr.loc[hr_cnt,'timestamp']
        filt_time = df_filt.loc[i,'timestamp']

        if(hr_time<=filt_time):
            if(hr_cnt<len(df_hr)-1):
                hr_cnt += 1
        df_filt.loc[i,'HR'] = df_hr.loc[hr_cnt,'hr']

    # Normalize by dividing by g (standard gravity)
    g = 9.8
    df_filt.loc[:,'x'] = df_filt['x'].apply(lambda x: x/g)
    df_filt.loc[:,'y'] = df_filt['y'].apply(lambda x: x/g)
    df_filt.loc[:,'z'] = df_filt['z'].apply(lambda x: x/g)
    
    cols = ['x','y','z']
    xyz_ = df_filt[cols].to_dict('split')['data']
    xyz_new = MinMaxScaler().fit_transform(xyz_)
#     print(np.array(xyz_new).shape)

    for i in range(len(cols)):
        df_filt[cols[i]] = pd.Series(xyz_new.transpose()[i])
        
#     print(df_filt['x'])

    return df_filt

# Calculate Activity Index

In [13]:
std_i_bar = [0.00349329,0.00465817,0.00543154]
std_i_bar = np.array(std_i_bar)

In [14]:
def equation_bai(X_i):
    all_std = []
    
    std_i = np.std(X_i,axis=0)
    diff_std = std_i**2 - std_i_bar**2
    diff_std = (diff_std + 1) / (std_i_bar**2 + 1)
    
    diff_std_ = std_i**2

    all_std.append(diff_std)
    
    all_std = np.array(all_std)
    
    ai = np.sum(all_std**2,axis=1)/3
    ai[ai<0] = 0
    ai = np.sqrt(ai)
    
    return ai

In [15]:
def calc_ai(df1):
    H = 10
    ai1 = []

    for i in range(len(df1)):
        xyz_val = []
        if(i-H>=0):
            for j in range(H,0,-1):
                xyz_val.append([df1.loc[i-j,'x'],df1.loc[i-j,'y'],df1.loc[i-j,'z']])
            ai_val = float(equation_bai(xyz_val))
            ai1.append(ai_val)
        else:
            ai1.append(1)

    return ai1

# Colors for Each Acitivity

In [17]:
def prepare_color_labels(ts_, labels):
  
    accum = 0
    ts = []
    for x in ts_:
        accum += x
        ts.append(round(accum,3))

    lb_set = set()
    for x in labels:
        lb_set.add(x)

    lb_ = list(lb_set)

    set_cnt = []
    for i in range(len(lb_)):
        set_cnt.append(0)

    lb = []
    lb.append('NaN')

    for x in labels:
        for i in range(len(lb_)):
            if(lb_[i]==x and set_cnt[i]!=1 and lb_[i]!='NaN'):
                set_cnt[i] = 1
                lb.append(x)

    colors = ['#808080', '#E6194B', '#3CB44B', '#FFE119', '#4363D8', '#F58231',
            '#911EB4', '#46F0F0', '#F032E6', '#BCF60C', '#008080', '#E6BEFF', 
            '#9A6324', '#800000', '#AAFFC3', '#808000', '#000075']

    color_dict = {}
    for i in range(len(lb)):
        color_dict[lb[i]] = colors[i]

    #   print(color_dict)

    lb_color = []
    for x in labels:
        lb_color.append(color_dict[x])

    return ts, lb_color

# Dataframe List Grouped by Label

In [110]:
def group_dataframe_by_label(df1, df_timer, subject_id, label_list):
    df_list = {}
    
    for label in label_list:
        df_list[label] = pd.DataFrame()
    
    for label in label_list:
        print(label)
        for i in range(df_timer.shape[0]):
            start = 0
            end = 0
            
            if(df_timer.loc[i, 'label']==label):
                t_a = df_timer.loc[i, 'time_start']
                t_b = df_timer.loc[i, 'time_end']

                for j in range(df1.shape[0]):    
                    if(df1.loc[j, 'ID']==subject_id):
                        if(j>0 and df1.loc[j, 'timestamp']<=t_b and df1.loc[j-1, 'timestamp']<t_b):
                            end = j

                for j in reversed(range(df1.shape[0])):
                    if(df1.loc[j, 'ID']==subject_id):
                        if(j<df1.shape[0]-1 and df1.loc[j, 'timestamp']>=t_a and df1.loc[j+1, 'timestamp']>t_a):
                            start = j

    #             print(t_a, t_b)
#                 print(start, end)

#                 print(df1[start:end])
                df_list[label].append(df1[start:end])

    for label in label_list:
        df_list[label] = df_list[label].reset_index(drop=True)

    return df_list

In [20]:
def separate_label(df1, df_timer, df_list, labels):
  
    df_label = df_timer.groupby('label')
    td_col = ['timestamp','duration']
    cols = ['x','y','z']

    for x in df_label:
    # x[0] is a label
    # x[1] is a groupby object
    
        df_label_x = df_label.get_group(x[0])
        df_label_x = df_label_x.reset_index(drop=True)

        temp_ts = [[df_label_x.loc[a]['timestamp'].split(' ')[1], 
                    calc_ts( calc_sec(df_label_x.loc[a]['timestamp'].split(' ')[1])+
                            calc_sec(df_label_x.loc[a]['duration']) )] 
                    for a in range(len(df_label_x))]
        
        for a in temp_ts:
            filter_ = [i for i in df1['timestamp'] 
                    if calc_sec(i.split(' ')[1])>=calc_sec(a[0]) and calc_sec(i.split(' ')[1])<=calc_sec(a[1])]

            df1_new = df1[df1['timestamp'].isin(filter_)]
            df1_new = df1_new.reset_index(drop=True)
            
#             xyz_ = df1_new[cols].to_dict('split')['data']
#             xyz_new = MinMaxScaler().fit_transform(xyz_)
            
#             for i in range(len(cols)):
#                 df1_new[cols[i]] = pd.Series(xyz_new.transpose()[i])

            for i in range(len(labels)):
                if(labels[i]==x[0]):
                    df_list[i] = df_list[i].append(df1_new, sort=False)

    return df_list    

# Get X and y from Dataset for Each Subject

In [23]:
def get_data(df_list, label_dict):
    feature_cols = ['x','y','z']
    count = 0
    
    y_all = []
    ts_all = []
    hr_all = []
    
    for x in label_dict:
#         print(x)
    
        X_series = df_list[label_dict[x]][feature_cols]

        X_ = X_series.values.reshape((len(X_series),3))
        y_ = np.array([label_dict[x] for i in range(len(df_list[label_dict[x]]))])
        ts_ = np.array(df_list[label_dict[x]]['timestamp'])
        hr_ = np.array(df_list[label_dict[x]]['HR'])

          # 'downstairs': 0,
          # 'sit': 1,
          # 'sleep': 2,
          # 'stand': 3,
            
        y_all.append(y_)
        ts_all.append(ts_)
        hr_all.append(hr_)
        
        if(count==0):
            X_all = X_
            count += 1

        else:
            X_all = np.vstack((X_all, X_))

    y_all = np.hstack(y_all)
    ts_all = np.hstack(ts_all)
    hr_all = np.hstack(hr_all)
    
    return np.array(X_all), np.array(y_all), np.array(ts_all), np.array(hr_all)

In [24]:
def get_sorted_data(X_i, y_i, ts_i, hr_i, subj_i):
    df_ = pd.DataFrame({
        'ID': subj_i,
        'timestamp': ts_i,
        'x': [x[0] for x in X_i],
        'y': [x[1] for x in X_i],
        'z': [x[2] for x in X_i],
        'HR': hr_i,
        'label': y_i
    })
    
    df_sorted = df_.sort_values(by=['timestamp'])
    
    cols = ['x','y','z']
    X_i = df_sorted[cols].values.tolist()
    y_i = df_sorted['label'].values.tolist()
    ts_i = df_sorted['timestamp'].values.tolist()
    hr_i = df_sorted['HR'].values.tolist()
    subj_i = df_sorted['ID'].values.tolist()
    
    return X_i, y_i, ts_i, hr_i, subj_i

# Function Call *

In [99]:
def load_all_data(all_subjects):
    
    for subject_id in all_subjects:
        print("Loading {0}'s data".format(subject_id))

        df_timer = load_timer(subject_id)
        
        start_time = df_timer.loc[0, 'time_start']
        end_time = df_timer.loc[df_timer.shape[0]-1, 'time_end']
        
        df_filt = load_acc(subject_id, start_time, end_time)
        df_hr = load_hr(subject_id, start_time, end_time)

        df1 = merge_acc_and_hr(df_filt, df_hr)
        ai1 = calc_ai(df1)

        df1['AI'] = pd.Series(ai1)
#         print(df1)

        label_list = ['sit', 'sleep', 'stand', 'walk']
        
        # get a list of dataframe in which there are 4 types of activity
        df_list = group_dataframe_by_label(df1, df_timer, subject_id, label_list)

        for label in label_list:
            print(df_list[label])

In [109]:
load_all_data(all_subjects)

Loading 1001's data
sit
229 415
       ID               timestamp         x         y         z          HR  \
229  1001 2019-01-14 14:53:07.834  0.346669  0.532096  0.427995  105.820340   
230  1001 2019-01-14 14:53:07.996  0.345733  0.531776  0.429373  105.820340   
231  1001 2019-01-14 14:53:08.156  0.346329  0.532523  0.428860  105.820340   
232  1001 2019-01-14 14:53:08.318  0.345287  0.527082  0.428732  105.820340   
233  1001 2019-01-14 14:53:08.477  0.345031  0.527936  0.426841  105.820340   
234  1001 2019-01-14 14:53:08.637  0.344861  0.526335  0.426841  105.274170   
235  1001 2019-01-14 14:53:08.798  0.344011  0.528149  0.425944  105.274170   
236  1001 2019-01-14 14:53:08.959  0.344861  0.528149  0.424277  105.274170   
237  1001 2019-01-14 14:53:09.119  0.344861  0.526975  0.424406  105.274170   
238  1001 2019-01-14 14:53:09.280  0.344606  0.526975  0.425046  105.274170   
239  1001 2019-01-14 14:53:09.441  0.344521  0.526869  0.426456  105.274170   
240  1001 2019-01-14

1024 1210
        ID               timestamp         x         y         z         HR  \
1024  1001 2019-01-14 14:55:15.595  0.424196  0.701149  0.672275  62.713055   
1025  1001 2019-01-14 14:55:15.756  0.415348  0.699123  0.701756  62.713055   
1026  1001 2019-01-14 14:55:15.916  0.418921  0.707017  0.688425  62.713055   
1027  1001 2019-01-14 14:55:16.077  0.418751  0.705203  0.685862  62.713055   
1028  1001 2019-01-14 14:55:16.238  0.417220  0.702856  0.701756  62.713055   
1029  1001 2019-01-14 14:55:16.404  0.406500  0.680561  0.743799  69.233280   
1030  1001 2019-01-14 14:55:16.559  0.469968  0.676401  0.634333  69.233280   
1031  1001 2019-01-14 14:55:16.721  0.510890  0.737819  0.511280  69.233280   
1032  1001 2019-01-14 14:55:16.881  0.531649  0.860816  0.338012  69.233280   
1033  1001 2019-01-14 14:55:17.043  0.484176  0.932715  0.381850  69.233280   
1034  1001 2019-01-14 14:55:17.202  0.492428  0.883857  0.394283  69.233280   
1035  1001 2019-01-14 14:55:17.365  0.5170

0 186
       ID               timestamp         x         y         z          HR  \
0    1001 2019-01-14 14:52:31.035  0.232240  0.684961  0.471288   86.937775   
1    1001 2019-01-14 14:52:31.193  0.231985  0.686028  0.469237   86.937775   
2    1001 2019-01-14 14:52:31.356  0.232240  0.686242  0.470903   86.937775   
3    1001 2019-01-14 14:52:31.514  0.232751  0.686135  0.470903   86.937775   
4    1001 2019-01-14 14:52:31.675  0.232751  0.685495  0.471928   87.511505   
5    1001 2019-01-14 14:52:31.836  0.233091  0.686562  0.471288   87.511505   
6    1001 2019-01-14 14:52:31.997  0.232240  0.686775  0.472441   87.511505   
7    1001 2019-01-14 14:52:32.158  0.232240  0.686028  0.471288   87.511505   
8    1001 2019-01-14 14:52:32.318  0.229858  0.685068  0.473979   87.511505   
9    1001 2019-01-14 14:52:32.479  0.230368  0.685068  0.468211   87.511505   
10   1001 2019-01-14 14:52:32.640  0.231730  0.686988  0.472954   87.511505   
11   1001 2019-01-14 14:52:32.800  0.231134  0

1709 1900
        ID               timestamp         x         y         z          HR  \
1709  1001 2019-01-14 14:57:05.693  0.325634  0.695149  0.667404   92.946655   
1710  1001 2019-01-14 14:57:05.852  0.332015  0.700909  0.621900   92.946655   
1711  1001 2019-01-14 14:57:06.015  0.309299  0.708057  0.679324   92.946655   
1712  1001 2019-01-14 14:57:06.174  0.304109  0.726191  0.675479   92.946655   
1713  1001 2019-01-14 14:57:06.336  0.306321  0.646185  0.649202   92.946655   
1714  1001 2019-01-14 14:57:06.497  0.308703  0.630610  0.579728   87.233734   
1715  1001 2019-01-14 14:57:06.658  0.292198  0.649492  0.585368   87.233734   
1716  1001 2019-01-14 14:57:06.816  0.264378  0.649812  0.568064   87.233734   
1717  1001 2019-01-14 14:57:06.977  0.251531  0.659093  0.613440   87.233734   
1718  1001 2019-01-14 14:57:07.138  0.246257  0.652799  0.602160   87.233734   
1719  1001 2019-01-14 14:57:07.298  0.256126  0.659839  0.541402   87.233734   
1720  1001 2019-01-14 14:57:07

3425 3505
        ID               timestamp         x         y         z          HR  \
3425  1001 2019-01-14 15:01:41.484  0.146057  0.570766  0.476831   92.018920   
3426  1001 2019-01-14 15:01:41.646  0.302089  0.690562  0.438249   91.807846   
3427  1001 2019-01-14 15:01:41.805  0.263293  0.643625  0.438634   91.807846   
3428  1001 2019-01-14 15:01:41.966  0.196678  0.680854  0.483625   91.807846   
3429  1001 2019-01-14 15:01:42.127  0.305747  0.675841  0.482215   91.807846   
3430  1001 2019-01-14 15:01:42.288  0.186383  0.657493  0.429533   91.807846   
3431  1001 2019-01-14 15:01:42.448  0.216926  0.645118  0.447094   91.807846   
3432  1001 2019-01-14 15:01:42.610  0.298090  0.632104  0.479010   91.807846   
3433  1001 2019-01-14 15:01:42.772  0.236154  0.505907  0.346376   91.901390   
3434  1001 2019-01-14 15:01:42.931  0.233091  0.481905  0.332276   91.901390   
3435  1001 2019-01-14 15:01:43.091  0.369810  0.666133  0.476319   91.901390   
3436  1001 2019-01-14 15:01:43

...

1569 1755
        ID               timestamp         x         y         z         HR  \
1569  3006 2019-01-16 14:02:59.981  0.859147  0.344818  0.787722  74.238940   
1570  3006 2019-01-16 14:03:00.143  0.864409  0.344623  0.782801  74.238940   
1571  3006 2019-01-16 14:03:00.302  0.861486  0.338598  0.781571  74.238940   
1572  3006 2019-01-16 14:03:00.463  0.861681  0.341902  0.783785  74.238940   
1573  3006 2019-01-16 14:03:00.624  0.861096  0.341902  0.783047  74.238940   
1574  3006 2019-01-16 14:03:00.786  0.863435  0.339181  0.781571  74.953460   
1575  3006 2019-01-16 14:03:00.945  0.862266  0.339764  0.782555  74.953460   
1576  3006 2019-01-16 14:03:01.106  0.861681  0.339958  0.778618  74.953460   
1577  3006 2019-01-16 14:03:01.267  0.862266  0.339181  0.781817  74.953460   
1578  3006 2019-01-16 14:03:01.427  0.861291  0.341708  0.781079  74.953460   
1579  3006 2019-01-16 14:03:01.587  0.860512  0.345012  0.785016  74.953460   
1580  3006 2019-01-16 14:03:01.749  0.8626

455 641
       ID               timestamp         x         y         z         HR  \
455  3006 2019-01-16 14:00:01.061  0.253398  0.449585  0.547641  60.324863   
456  3006 2019-01-16 14:00:01.203  0.250280  0.455416  0.545181  60.324863   
457  3006 2019-01-16 14:00:01.363  0.252229  0.450556  0.548133  60.324863   
458  3006 2019-01-16 14:00:01.525  0.249890  0.449196  0.544442  60.069252   
459  3006 2019-01-16 14:00:01.684  0.255152  0.450556  0.539521  60.069252   
460  3006 2019-01-16 14:00:01.846  0.254568  0.452500  0.547395  60.069252   
461  3006 2019-01-16 14:00:02.008  0.250865  0.450751  0.547887  60.069252   
462  3006 2019-01-16 14:00:02.173  0.250865  0.451723  0.549855  60.069252   
463  3006 2019-01-16 14:00:02.326  0.251839  0.449196  0.544688  60.069252   
464  3006 2019-01-16 14:00:02.490  0.252424  0.453666  0.545181  60.055195   
465  3006 2019-01-16 14:00:02.648  0.250475  0.454249  0.549363  60.055195   
466  3006 2019-01-16 14:00:02.809  0.253203  0.450362  0

2144 2336
        ID               timestamp         x         y         z         HR  \
2144  3006 2019-01-16 14:04:32.355  0.246821  0.473881  0.548564  77.546080   
2145  3006 2019-01-16 14:04:32.516  0.258124  0.468050  0.533801  76.688920   
2146  3006 2019-01-16 14:04:32.677  0.259488  0.479518  0.526235  76.688920   
2147  3006 2019-01-16 14:04:32.837  0.217393  0.475630  0.527219  76.688920   
2148  3006 2019-01-16 14:04:32.998  0.253447  0.487293  0.524266  76.688920   
2149  3006 2019-01-16 14:04:33.160  0.263971  0.494096  0.545857  76.688920   
2150  3006 2019-01-16 14:04:33.322  0.208234  0.489625  0.554715  76.688920   
2151  3006 2019-01-16 14:04:33.480  0.283264  0.490597  0.560374  76.688920   
2152  3006 2019-01-16 14:04:33.642  0.247601  0.462413  0.572430  76.260345   
2153  3006 2019-01-16 14:04:33.802  0.264945  0.474076  0.557667  76.260345   
2154  3006 2019-01-16 14:04:33.962  0.227917  0.463385  0.523282  76.260345   
2155  3006 2019-01-16 14:04:34.122  0.2705

3093 3359
        ID               timestamp         x         y         z         HR  \
3093  3006 2019-01-16 14:07:04.821  0.232010  0.487098  0.561604  65.377190   
3094  3006 2019-01-16 14:07:04.982  0.230646  0.459886  0.623608  65.377190   
3095  3006 2019-01-16 14:07:05.142  0.255201  0.490792  0.575629  65.377190   
3096  3006 2019-01-16 14:07:05.304  0.246041  0.470382  0.594821  65.377190   
3097  3006 2019-01-16 14:07:05.464  0.242339  0.486904  0.582764  65.377190   
3098  3006 2019-01-16 14:07:05.624  0.240780  0.493513  0.591376  65.377190   
3099  3006 2019-01-16 14:07:05.785  0.252278  0.510035  0.575629  65.359300   
3100  3006 2019-01-16 14:07:05.945  0.262022  0.443559  0.576121  65.359300   
3101  3006 2019-01-16 14:07:06.107  0.256175  0.487293  0.561112  65.359300   
3102  3006 2019-01-16 14:07:06.267  0.263581  0.486515  0.566033  65.359300   
3103  3006 2019-01-16 14:07:06.427  0.236297  0.491569  0.581534  65.359300   
3104  3006 2019-01-16 14:07:06.589  0.2277

3708 3937
        ID               timestamp         x         y         z         HR  \
3708  3006 2019-01-16 14:08:44.966  0.260171  0.482045  0.594575  94.553510   
3709  3006 2019-01-16 14:08:45.126  0.176175  0.471743  0.551762  94.553510   
3710  3006 2019-01-16 14:08:45.287  0.271474  0.515283  0.536507  94.553510   
3711  3006 2019-01-16 14:08:45.451  0.207162  0.492347  0.549056  94.879400   
3712  3006 2019-01-16 14:08:45.609  0.232692  0.471937  0.539952  94.879400   
3713  3006 2019-01-16 14:08:45.769  0.256857  0.484960  0.562096  94.879400   
3714  3006 2019-01-16 14:08:45.930  0.224312  0.474076  0.579074  94.879400   
3715  3006 2019-01-16 14:08:46.090  0.318051  0.494290  0.595805  94.879400   
3716  3006 2019-01-16 14:08:46.251  0.215737  0.413626  0.577597  94.879400   
3717  3006 2019-01-16 14:08:46.413  0.174421  0.440838  0.541182  94.879400   
3718  3006 2019-01-16 14:08:46.571  0.187674  0.477380  0.537245  95.042340   
3719  3006 2019-01-16 14:08:46.735  0.2570